## Arrays & Hashing

### Medium

[36. Valid Sudoku](https://leetcode.com/problems/valid-sudoku/)

In [54]:
from collections import defaultdict 
#36. Valid Sudoku
def isValidSudoku(board: list[list[str]]) -> bool:
    #Idea: check if each small square belongs to coordinated-associated rows, cols, and sqr 
    #Implement: create hashset (dict of sets) as cols, rows and sqr
    #each row, col, sqr is a set
    #Form of each dict(set): e.g: rows = {0: {set}, 1:{set},...}
    cols = defaultdict(set) 
    rows = defaultdict(set)
    sqr = defaultdict(set)

    for r in range(9):
            for c in range(9):
                if board[r][c] == ".":
                    continue
                if (
                    board[r][c] in rows[r]
                    or board[r][c] in cols[c]
                    or board[r][c] in sqr[(r // 3, c // 3)]
                ):
                    return False
                cols[c].add(board[r][c])
                rows[r].add(board[r][c])
                sqr[(r // 3, c // 3)].add(board[r][c])

    return True

[238. Product of Array Except Self - Pre and pos as arrays](https://leetcode.com/problems/product-of-array-except-self/submissions/1089034138/)

In [55]:
def productExceptSelf(nums: list[int]) -> list[int]:
    #Create 2 arrays: prefix product and postfix product.
    #Output[i] = prefix[i-1] * postfix [i+1]
    #Time: O(n), Space: O(3n)
    l = len(nums)
    pre, pos= [nums[0]]*l, [nums[l-1]]*l
    j = l - 2
    for i in range(1, l):
        pre[i] = pre[i-1] * nums[i]
        pos[j] = pos[j+1] * nums[j]
        j-=1
    #init first and last in output
    out = [pos[1]]*l
    out[l-1] = pre[l-2]

    for i in range(1,l-1):
        out[i] = pre[i-1]*pos[i+1]

    print(pre, pos, out)
    return out

nums = [1,2,3,4]
productExceptSelf(nums)

[1, 2, 6, 24] [24, 24, 12, 4] [24, 12, 8, 6]


[24, 12, 8, 6]

[238. Product of Array Except Self (2) - Only pos var](https://leetcode.com/problems/product-of-array-except-self/submissions/1089079628/)

In [56]:
def productExceptSelf(nums: list[int]) -> list[int]:
    #Create an preArr stores prefix product up to + (exclude) nums[i]
    #Loop from the end of preArr: at each preArr[i], multiply by current postfix product var

    #Time: O(n), Space: O(n)
    l = len(nums)
    out = [1] * l
    
    #Create an arr stores prefix product up to nums[i] + (exclude) nums[i]
    for i in range(1, l):
        out[i] = out[i-1] * nums[i-1]
    #init first and last in output
    pos = nums[l-1]
    for i in range(l-2, -1, -1):
        out[i] = out[i] * pos  #upto prefix * upto postfix 
        pos *= nums[i]

    print(out)
    return out

[128. Longest Consecutive Sequence. Original](https://leetcode.com/problems/longest-consecutive-sequence/)

In [57]:
def longestConsecutive(nums: list[int]) -> int:
    #Sort and count the highest
    nums.sort()
    if len(nums) == 0:
        return 0
    max = 1
    temp = 1
    for i in range(1, len(nums)):
        if nums[i] == nums[i-1]+1: 
            temp+=1
        elif nums[i] == nums[i-1]:
            continue
        else:
            temp = 1 #reset
        if temp > max:
            max = temp
    print(max)
    return max

[347. Top K Frequent Elements ](https://leetcode.com/problems/top-k-frequent-elements/submissions/)

Bucket sort - Space: Linear, Time: Linear

In [58]:
def topKFrequent(nums: list[int], k: int) -> list[int]:
#bucket sort: create an array where index is the freqs, value is list of nums associating with the freqs
# Space: O(N). Size of bucket always <= len(nums) because max freq of any num is len(nums).
    count = {} # count dict
    
    buckets = [[] for i in range (len(nums)+1)] #freqs start from 1 to including len(nums)
    
    print(buckets)
    #manually count instead of collections.Counter(nums)
    for n in nums:
        count[n] = 1 + count.get(n, 0) #count.get(n, 0) means count[n] = 0 if n has not exist in dict
    
    #bucket's index is the freqs, value is list of nums associating with the freqs
    for num,freq in count.items():
        
        buckets[freq].append(num)
    res = []
    for i in range(len(buckets)-1, 0, -1): #start from last index
        if len(buckets[i]) == 0:
            continue
        for num in buckets[i]:
            res.append(num)
            if len(res) == k:
                return res
    return res

Heap - O(KlogN)

In [59]:
def topKFrequent(nums: list[int], k: int) -> list[int]:
    count = Counter(nums)
    #create a list of tuples (fre, num) for heapifying (we maintain heap by values)
    heap = [(fre, num) for (num, fre) in count.items() ]
    
    heapq._heapify_max(heap)
    print(heap)

    mlist = []
    for i in range(k):
        mlist.append(heapq._heappop_max(heap)[1]) #get the num (heap[1])
    return mlist

## Two Pointers

### Medium

[167. Two Sum II - Input Array Is Sorted](https://leetcode.com/problems/two-sum-ii-input-array-is-sorted/)

In [60]:
def twoSum(numbers: list[int], target: int) -> list[int]:
    #2 pointers: at start and at end. 
    # When sum(nums[i]+nums[j])> target -> decrement end p (decrease larger num in the sum)
    #sum(nums[i]+nums[j]) < target -> increase start p 
    #Time: O(n), Space: O(1)

    pre, pos = 0, len(numbers)-1
    while pre < pos:
        sum = numbers[pre]+numbers[pos]
        if sum < target:
            pre += 1
        elif sum > target:
            pos -= 1
        else:
            return [pre+1, pos+1]
    return []

[15. 3Sum](https://leetcode.com/problems/3sum/)

In [61]:
def threeSum(nums: list[int]) -> list[list[int]]:
    nums.sort()
    res = []

    #i is pivot
    for i in range(len(nums)):
        #no repeated triplet
        if i > 0 and nums[i] == nums[i-1]:
            continue 
        #can't offset from positive pivot 
        if nums[i] > 0:
            break
        l, r = i+1, len(nums)-1
        while l < r:
            
            sum = nums[i]+nums[l]+nums[r]
            
            if sum == 0:
                res.append([nums[i],nums[l],nums[r]])
                #only need to update 1 pointer
                l+=1 
                #no repeated triplet
                while nums[l-1] == nums[l] and l<r :
                    l+=1
                
            elif sum > 0:
                r-=1
            elif sum < 0:
                l+=1
        
    return res
nums = [-1,0,1,2,-1,-4]
nums1= [-2,0,1,1,2]
display(threeSum(nums1))
display(threeSum(nums))

[[-2, 0, 2], [-2, 1, 1]]

[[-1, -1, 2], [-1, 0, 1]]

## Stack

### Easy

[20. Valid Parentheses](https://leetcode.com/problems/valid-parentheses/)

In [62]:
def isValid(s: str) -> bool:
    brack = {'{':'}', '[':']', '(':')'}
    stack = []
    for i in range(len(s)):
        if s[i] in brack.keys():
            stack.append(s[i])
        elif len(stack) != 0:
            if s[i] != brack[stack[-1]]: #peek
                return False
            stack.pop()
        # len(stack) == 0 + still have non-open char -> illegal close char
        elif len(stack) == 0: 
            return False
    if len(stack) != 0:
        return False
    return True

### Medium

[155. Min Stack](https://leetcode.com/problems/min-stack/)

In [63]:
class MinStack:
    #Create a another minstack next to normal stack that stores upto current min val
    def __init__(self):
        self.stack = []
        self.minSt = []

    def push(self, val: int) -> None:
        self.stack.append(val)
        #temp min
        # Only consider minSt[-1] if minSt not EMPTY
        self.minSt.append(min(val, self.minSt[-1] if self.minSt else val))


    def pop(self) -> None:
        #pop minSt always true because it pop the same as stack IF stack[i] is the min value at its temp time.
        #else, it pop the temporary min
        self.minSt.pop()
        self.stack.pop()

    def top(self) -> int:
        return self.stack[-1]

    def getMin(self) -> int:
        return self.minSt[-1]


min_stack = MinStack()

# Push elements onto the stack
min_stack.push(3)
min_stack.push(1)

# Get the current minimum element
min_element = min_stack.getMin()
print(f"Minimum element in the stack: {min_element}")  # Output: Minimum element in the stack: 1

Minimum element in the stack: 1


[150. Evaluate Reverse Polish Notation](https://leetcode.com/problems/evaluate-reverse-polish-notation/)

This is postfix travel, utilize stack. Pop nums when encounter sign. Then push the result back to stack

In [64]:
import math
def evalRPN(tokens: list[str]) -> int:
    st = []
    res = 0
    for i in tokens:
        ch = i
        if ch == '+':
            res = st.pop() + st.pop()
        elif ch == '-':
            #later digit on top, pop first
            b,a = st.pop(), st.pop()
            res = a - b
        elif ch == '*':
            res = st.pop() * st.pop()
        elif ch == '/':
            #cast to int
            #e.g: -0.5 = 0, 8.9=8
            b,a = st.pop(), st.pop()
            res = int(a/b)
        else: #if num
            res = int(i)
        st.append(res)
    return st.pop()
tokens = ["4","13","5","/","+"]
evalRPN(tokens)



6

[22. Generate Parentheses](https://leetcode.com/problems/generate-parentheses/)

[Generate Parentheses - Stack - Leetcode 22](https://www.youtube.com/watch?v=s9fokUqJ76A&t=1s)

<img src="assets/l22.png" width=800px />

In [1]:
def generateParenthesis(n: int) -> list[str]:
    out = []
    comb = [] #each combination
    #Learned recursion TREE with # of o and c parentheses from 0 to n
    #add ( when open < n, add ) when close < open
    
    def backtrack(ocount, ccount):
        #base case
        if ocount == n and ccount == n:
            out.append("".join(comb))
        
        if ocount < n:
            comb.append('(')
            backtrack(ocount+1, ccount)
            comb.pop()
        if ccount < ocount: 
            comb.append(')')
            backtrack(ocount, ccount+1)
            comb.pop()

    
    backtrack(0,0)
    return out
n=3
generateParenthesis(n)

['((()))', '(()())', '(())()', '()(())', '()()()']

[739. Daily Temperatures](https://leetcode.com/problems/daily-temperatures/)

In [25]:
def dailyTemperatures(temperatures: list[int]) -> list[int]:
    
    #Pattern reg: find nearest correlation -> stack.pop get the most recent val within the correlation.
    #Add temps to stacks, only pop when found the nearest warm.
    
    st = {} #val:count stack
    out = [0]*len(temperatures)
    ist = [] #index stack correspond to val in st in output.
    
    for i,t in enumerate(temperatures):
        
        if len(st) > 0:
            while len(st) > 0 and t > st[list(st)[-1]]:
                
                out[ist.pop()] = st.pop(list(st)[-1]) # st.keys()[-1] = key of top stack, dict.pop(key) pops and return val 
            for k,v in st.items():
                st[k]= v+1
                
        st.update({t: 1})
        ist.append(i)
    return out

temperatures = [73,74,75,71,69,72,76,73]
print(dailyTemperatures(temperatures))

KeyboardInterrupt: 